In [450]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import math
import json
import re
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F

import random
np.random.seed(42)
random.seed(42)

# Data

In [451]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 1234

# Python & NumPy
random.seed(SEED)
np.random.seed(SEED)

# PyTorch
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Determinism flags
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [452]:
encoder_input = torch.randn(1, 512, device=device)
bottleneck_input = torch.randn(256, 32, device=device)
decoder_first_input = torch.randn(256, 32, device=device)
decoder_last_input = torch.randn(64, 256, device=device)
out_input = torch.randn(16, 512, device=device)

# Model Definition

In [453]:
class ConvBlock1D(nn.Module):
    """
    Conv1d -> BatchNorm1d -> Activation
    """
    def __init__(self, in_ch, out_ch, k=15, s=2, p=7, bias=True, act="lrelu"):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size=k, stride=s, padding=p, bias=bias)
        self.norm = nn.BatchNorm1d(out_ch)

        if act == "lrelu":
            self.act = nn.LeakyReLU(0.2, inplace=True)
        elif act == "relu":
            self.act = nn.ReLU(inplace=True)
        else:
            raise ValueError("act must be 'lrelu' or 'relu'")

    def forward(self, x):
        return self.act(self.norm(self.conv(x)))


class DeconvBlock1D(nn.Module):
    """
    ConvTranspose1d -> BatchNorm1d -> Activation
    """
    def __init__(self, in_ch, out_ch, k=4, s=2, p=1, bias=True, act="relu"):
        super().__init__()
        self.deconv = nn.ConvTranspose1d(in_ch, out_ch, kernel_size=k, stride=s, padding=p, bias=bias)
        self.norm = nn.BatchNorm1d(out_ch)

        if act == "relu":
            self.act = nn.ReLU(inplace=True)
        elif act == "lrelu":
            self.act = nn.LeakyReLU(0.2, inplace=True)
        else:
            raise ValueError("act must be 'relu' or 'lrelu'")

    def forward(self, x):
        return self.act(self.norm(self.deconv(x)))


class ResBlock1D(nn.Module):
    """
    Residual block: (Conv -> BN -> ReLU) x2 + skip
    Keeps same channel count and length.
    """
    def __init__(self, ch, k=7, p=3, bias=True):
        super().__init__()
        self.c1 = nn.Conv1d(ch, ch, kernel_size=k, stride=1, padding=p, bias=bias)
        self.n1 = nn.BatchNorm1d(ch)
        self.c2 = nn.Conv1d(ch, ch, kernel_size=k, stride=1, padding=p, bias=bias)
        self.n2 = nn.BatchNorm1d(ch)

    def forward(self, x):
        h = F.relu(self.n1(self.c1(x)))
        h = self.n2(self.c2(h))
        return F.relu(x + h)
    
class MultiScaleResBlock1D(nn.Module):
    """
    Multi-scale residual block: parallel conv branches (k=3,5,7) then fuse.
    Keeps same channel count and length.
    """
    def __init__(self, ch, bias=True):
        super().__init__()

        self.b3 = nn.Sequential(
            nn.Conv1d(ch, ch, kernel_size=3, padding=1, bias=bias),
            nn.BatchNorm1d(ch),
            nn.ReLU(inplace=True),
        )
        self.b5 = nn.Sequential(
            nn.Conv1d(ch, ch, kernel_size=5, padding=2, bias=bias),
            nn.BatchNorm1d(ch),
            nn.ReLU(inplace=True),
        )
        self.b7 = nn.Sequential(
            nn.Conv1d(ch, ch, kernel_size=7, padding=3, bias=bias),
            nn.BatchNorm1d(ch),
            nn.ReLU(inplace=True),
        )

        self.fuse = nn.Sequential(
            nn.Conv1d(ch, ch, kernel_size=1, bias=bias),
            nn.BatchNorm1d(ch),
        )

    def forward(self, x):
        h = self.b3(x) + self.b5(x) + self.b7(x)
        h = self.fuse(h)
        return F.relu(x + h)



In [454]:
# NN Generator (U-Net-ish + Res bottleneck)
class GeneratorCNNWGAN(nn.Module):
    """
    CNN U-Net-ish generator for EEG denoising (WGAN).
    Input : (B, 1, 512) noisy_norm
    Output: (B, 1, 512) clean_norm_hat
    """
    def __init__(self, base_ch=32, bottleneck_blocks=4, bias=True):
        super().__init__()

        # Encoder
        self.e1 = ConvBlock1D(1, base_ch,       k=16, s=2, p=7, bias=bias, act="lrelu")      # 512 -> 256
        self.e2 = ConvBlock1D(base_ch, base_ch*2, k=16, s=2, p=7, bias=bias, act="lrelu")    # 256 -> 128
        self.e3 = ConvBlock1D(base_ch*2, base_ch*4, k=16, s=2, p=7, bias=bias, act="lrelu")  # 128 -> 64
        self.e4 = ConvBlock1D(base_ch*4, base_ch*8, k=16, s=2, p=7, bias=bias, act="lrelu")  # 64 -> 32

        # Bottleneck
        bn_ch = base_ch * 8
        self.bottleneck = nn.Sequential(*[
            ResBlock1D(bn_ch, k=7, p=3, bias=bias) for _ in range(bottleneck_blocks)
        ])

        # Decoder (concat doubles channels)
        self.d1 = DeconvBlock1D(bn_ch, base_ch*4,   k=4, s=2, p=1, bias=bias, act="relu")     # 32 -> 64
        self.d2 = DeconvBlock1D(base_ch*8, base_ch*2, k=4, s=2, p=1,bias=bias, act="relu")   # 64 -> 128
        self.d3 = DeconvBlock1D(base_ch*4, base_ch,   k=4, s=2, p=1, bias=bias, act="relu")   # 128 -> 256
        self.d4 = DeconvBlock1D(base_ch*2, base_ch//2, k=4, s=2, p=1, bias=bias, act="relu")  # 256 -> 512

        # Head (linear output recommended for normalized signals)
        self.out = nn.Conv1d(base_ch//2, 1, kernel_size=7, stride=1, padding=3, bias=bias)

    def forward(self, y):
        # Encoder
        s1 = self.e1(y)   # (B, base, 256)
        s2 = self.e2(s1)  # (B, 2b, 128)
        s3 = self.e3(s2)  # (B, 4b, 64)
        s4 = self.e4(s3)  # (B, 8b, 32)

        # Bottleneck
        b = self.bottleneck(s4)

        # Decoder + skip connections
        d1 = self.d1(b)                  # (B, 4b, 64)
        d1 = torch.cat([d1, s3], dim=1)  # (B, 8b, 64)

        d2 = self.d2(d1)                 # (B, 2b, 128)
        d2 = torch.cat([d2, s2], dim=1)  # (B, 4b, 128)

        d3 = self.d3(d2)                 # (B, b, 256)
        d3 = torch.cat([d3, s1], dim=1)  # (B, 2b, 256)

        d4 = self.d4(d3)                 # (B, b/2, 512)

        return self.out(d4)              # (B, 1, 512)
    
# Patch Critic (shared by CNN/ResCNN)
class CriticPatch1D(nn.Module):
    """
    Conditional PatchGAN critic for WGAN:
      D(y, x) -> patch scores
    y,x: (B,1,512)
    output: (B,1,32)
    """
    def __init__(self, base_ch=32, bias=True):
        super().__init__()
        self.c1 = nn.Conv1d(2, base_ch, kernel_size=16, stride=2, padding=7, bias=bias)  # 512 -> 256
        self.c2 = ConvBlock1D(base_ch, base_ch*2, k=16, s=2, p=7, bias=bias, act="lrelu")    # 256 -> 128
        self.c3 = ConvBlock1D(base_ch*2, base_ch*4, k=16, s=2, p=7, bias=bias, act="lrelu")  # 128 -> 64
        self.c4 = ConvBlock1D(base_ch*4, base_ch*8, k=16, s=2, p=7, bias=bias, act="lrelu")  # 64 -> 32
        self.out = nn.Conv1d(base_ch*8, 1, kernel_size=7, stride=1, padding=3, bias=bias)   # 32 -> 32

    def forward(self, y, x):
        h = torch.cat([y, x], dim=1)  # (B,2,512)
        h = F.leaky_relu(self.c1(h), 0.2, inplace=True)
        h = self.c2(h)
        h = self.c3(h)
        h = self.c4(h)
        return self.out(h)


In [455]:
class FusedConvBlock1D(nn.Module):
    def __init__(self, in_ch, out_ch, k, s, p, bias=True, act="lrelu"):
        super().__init__()
        self.conv = nn.Conv1d(in_ch, out_ch, k, s, p, bias=bias)

        if act == "lrelu":
            self.act = nn.LeakyReLU(0.2, inplace=True)
        elif act == "relu":
            self.act = nn.ReLU(inplace=True)
        else:
            raise ValueError

    def forward(self, x):
        return self.act(self.conv(x))


class FusedDeconvBlock1D(nn.Module):
    def __init__(self, in_ch, out_ch, k, s, p, bias=True, act="relu"):
        super().__init__()
        self.deconv = nn.ConvTranspose1d(in_ch, out_ch, k, s, p, bias=bias)

        if act == "relu":
            self.act = nn.ReLU(inplace=True)
        elif act == "lrelu":
            self.act = nn.LeakyReLU(0.2, inplace=True)
        else:
            raise ValueError

    def forward(self, x):
        return self.act(self.deconv(x))


In [456]:
class GeneratorCNNWGAN_Fused(nn.Module):
    def __init__(self, base_ch=32, bottleneck_blocks=4, bias=True):
        super().__init__()

        self.e1 = FusedConvBlock1D(1, base_ch, 16, 2, 7, bias, "lrelu")
        self.e2 = FusedConvBlock1D(base_ch, base_ch*2, 16, 2, 7, bias, "lrelu")
        self.e3 = FusedConvBlock1D(base_ch*2, base_ch*4, 16, 2, 7, bias, "lrelu")
        self.e4 = FusedConvBlock1D(base_ch*4, base_ch*8, 16, 2, 7, bias, "lrelu")

        self.bottleneck = nn.Sequential(*[
            nn.Sequential(
                nn.Conv1d(base_ch*8, base_ch*8, 7, 1, 3, bias=bias),
                nn.ReLU(inplace=True),
                nn.Conv1d(base_ch*8, base_ch*8, 7, 1, 3, bias=bias),
            )
            for _ in range(bottleneck_blocks)
        ])

        self.d1 = FusedDeconvBlock1D(base_ch*8, base_ch*4, 4, 2, 1, bias)
        self.d2 = FusedDeconvBlock1D(base_ch*8, base_ch*2, 4, 2, 1, bias)
        self.d3 = FusedDeconvBlock1D(base_ch*4, base_ch, 4, 2, 1, bias)
        self.d4 = FusedDeconvBlock1D(base_ch*2, base_ch//2, 4, 2, 1, bias)

        self.out = nn.Conv1d(base_ch//2, 1, 7, 1, 3, bias=bias)

    def forward(self, y):
        s1 = self.e1(y)
        s2 = self.e2(s1)
        s3 = self.e3(s2)
        s4 = self.e4(s3)

        b = s4
        for blk in self.bottleneck:
            b = F.relu(b + blk(b))

        d1 = torch.cat([self.d1(b), s3], dim=1)
        d2 = torch.cat([self.d2(d1), s2], dim=1)
        d3 = torch.cat([self.d3(d2), s1], dim=1)
        d4 = self.d4(d3)

        return self.out(d4)


In [457]:
def fuse_conv_bn_1d(
    conv_weight,        # (Cout, Cin, K)
    conv_bias,          # (Cout,) or None
    running_mean,       # (Cout,)
    running_var,        # (Cout,)
    bn_weight,          # (Cout,) or None (gamma)
    bn_bias,            # (Cout,) or None (beta)
    eps=1e-5
):
    Cout = conv_weight.shape[0]

    if bn_weight is None:
        bn_weight = torch.ones(Cout, device=conv_weight.device, dtype=conv_weight.dtype)
    if bn_bias is None:
        bn_bias = torch.zeros(Cout, device=conv_weight.device, dtype=conv_weight.dtype)
    if conv_bias is None:
        conv_bias = torch.zeros(Cout, device=conv_weight.device, dtype=conv_weight.dtype)

    denom = torch.sqrt(running_var + eps)          # (Cout,)
    scale = bn_weight / denom                      # (Cout,)

    # Fuse weight
    fused_weight = conv_weight * scale[:, None, None]

    # Fuse bias
    fused_bias = (conv_bias - running_mean) * scale + bn_bias

    return fused_weight, fused_bias

def fuse_deconv_bn_1d(
    deconv_weight,     # (Cin, Cout, K)
    deconv_bias,       # (Cout,) or None
    running_mean,      # (Cout,)
    running_var,       # (Cout,)
    bn_weight,         # (Cout,)
    bn_bias,           # (Cout,)
    eps
):
    Cin, Cout, K = deconv_weight.shape

    if deconv_bias is None:
        deconv_bias = torch.zeros(
            Cout,
            device=deconv_weight.device,
            dtype=deconv_weight.dtype
        )

    # BN scale
    scale = bn_weight / torch.sqrt(running_var + eps)  # (Cout,)

    # Fuse weights (scale on Cout dimension)
    fused_weight = deconv_weight * scale.view(1, Cout, 1)

    # Fuse bias
    fused_bias = (deconv_bias - running_mean) * scale + bn_bias

    return fused_weight, fused_bias


In [458]:
def fuse_generator(G):
    G_fused = GeneratorCNNWGAN_Fused(bias=True).to(device)
    G_fused.eval()

    with torch.no_grad():
        # Encoder
        for i in range(1, 5):
            e = getattr(G, f"e{i}")
            fe = getattr(G_fused, f"e{i}")

            w, b = fuse_conv_bn_1d(
                e.conv.weight, e.conv.bias,
                e.norm.running_mean, e.norm.running_var,
                e.norm.weight, e.norm.bias
            )
            fe.conv.weight.copy_(w)
            fe.conv.bias.copy_(b)

        # Bottleneck
        for i, blk in enumerate(G.bottleneck):
            fblk = G_fused.bottleneck[i]

            for j, (c, n) in enumerate([(blk.c1, blk.n1), (blk.c2, blk.n2)]):
                w, b = fuse_conv_bn_1d(
                    c.weight, c.bias,
                    n.running_mean, n.running_var,
                    n.weight, n.bias
                )
                fblk[j*2].weight.copy_(w)
                fblk[j*2].bias.copy_(b)

        # Decoder
        for i in range(1, 5):
            d = getattr(G, f"d{i}")
            fd = getattr(G_fused, f"d{i}")

            w, b = fuse_deconv_bn_1d(
                d.deconv.weight, d.deconv.bias,
                d.norm.running_mean, d.norm.running_var,
                d.norm.weight, d.norm.bias,
                d.norm.eps
            )
            fd.deconv.weight.copy_(w)
            fd.deconv.bias.copy_(b)

        # Output
        G_fused.out.weight.copy_(G.out.weight)
        G_fused.out.bias.copy_(G.out.bias)

    return G_fused


# Model Import

In [459]:
BIAS = True
DATA_MODE = 5 # up to 5

In [460]:
data_path = os.path.abspath(f"../models/main3_d{DATA_MODE}_{"b" if BIAS else "nb"}/")
print(data_path)

pattern = re.compile(r"^cnn_([DG])_\d{8}_\d{6}\.pth$")

cnn_G_path = None
cnn_D_path = None

for f in os.listdir(data_path):
    m = pattern.match(f)
    if m:
        full = os.path.join(data_path, f)
        if m.group(1) == "G":
            cnn_G_path = full
        else:
            cnn_D_path = full

print("G:", cnn_G_path)
print("D:", cnn_D_path)

c:\Users\Aryo\PersonalMade\Programming\GAN\repo\src\models\main3_d5_b
G: c:\Users\Aryo\PersonalMade\Programming\GAN\repo\src\models\main3_d5_b\cnn_G_20260114_024826.pth
D: c:\Users\Aryo\PersonalMade\Programming\GAN\repo\src\models\main3_d5_b\cnn_D_20260114_024826.pth


In [461]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

G = GeneratorCNNWGAN(bias=BIAS).to(device)
D = CriticPatch1D(bias=BIAS).to(device)

G.load_state_dict(torch.load(cnn_G_path, map_location=device))
D.load_state_dict(torch.load(cnn_D_path, map_location=device))

print(G.eval())
print(D.eval())

GeneratorCNNWGAN(
  (e1): ConvBlock1D(
    (conv): Conv1d(1, 32, kernel_size=(16,), stride=(2,), padding=(7,))
    (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (e2): ConvBlock1D(
    (conv): Conv1d(32, 64, kernel_size=(16,), stride=(2,), padding=(7,))
    (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (e3): ConvBlock1D(
    (conv): Conv1d(64, 128, kernel_size=(16,), stride=(2,), padding=(7,))
    (norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (e4): ConvBlock1D(
    (conv): Conv1d(128, 256, kernel_size=(16,), stride=(2,), padding=(7,))
    (norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.2, inplace=True)
  

C:\Users\Aryo\AppData\Local\Temp\ipykernel_3556\700924479.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  G.load_state_dict(torch.load(cnn_G_path, map_location=device))


In [462]:
G_fused = fuse_generator(G)

In [463]:
Q_CONFIGS = {
    "Q4.12": dict(frac_bits=12, int_bits=4, dtype=np.int16),
    "Q10.10": dict(frac_bits=10, int_bits=10, dtype=np.int32),
    "Q9.14": dict(frac_bits=14, int_bits=10, dtype=np.int32),
}

def float_to_q(x, frac_bits, int_bits, dtype):
    scale = 1 << frac_bits
    total_bits = int_bits + frac_bits
    min_val = -(1 << (total_bits - 1))
    max_val = (1 << (total_bits - 1)) - 1

    xq = np.round(x * scale)
    xq = np.clip(xq, min_val, max_val)
    return xq.astype(dtype)

def q_to_float(x, frac_bits):
    return x.astype(np.float32) / (1 << frac_bits)

In [464]:
TYPE =  "Q9.14" # "Q4.12", "Q10.10","Q9.14", or "FLOAT"

TIME_REPEAT = {
    "encoder":    [256, 128, 64, 32],
    "bottleneck": [32]*8,
    "decoder":    [64, 128, 256, 512],
    "out":        [512]
}

if TYPE == "FLOAT":
    MODE = "FLOAT"
    DTYPE = np.float32
else:
    MODE = "FIXED"
    cfg = Q_CONFIGS[TYPE]
    FRAC_BITS = cfg["frac_bits"]
    INT_BITS  = cfg["int_bits"]
    DTYPE     = cfg["dtype"]

# Mem Output

In [465]:
def classify_part(param_name):
    if param_name.startswith(("e1.", "e2.", "e3.", "e4.")):
        return "encoder"
    elif param_name.startswith("bottleneck."):
        return "bottleneck"
    elif param_name.startswith(("d1.", "d2.", "d3.", "d4.")):
        return "decoder"
    elif param_name.startswith(("out.")):
        return "out"
    else:
        raise ValueError(f"Unknown param group: {param_name}")
    
def is_bias(name):
    return name.endswith(".bias")

def is_weight(name):
    return name.endswith(".weight")

def expand_bias(bias, repeat_n):
    # bias shape: [C]
    # output shape: [C * repeat_n]
    return np.repeat(bias, repeat_n)

def flatten_conv_weight(w):
    # shape [x, y, z]
    return w.reshape(-1)

def flatten_deconv_weight(w):
    # Original shape: (x, y, z)
    # Target iteration order (Slowest to Fastest): y -> z -> x
    # This matches the indices: [0,0,0], [1,0,0], [0,0,1], [1,0,1]...
    w_perm = np.transpose(w, (1, 2, 0)) 
    return w_perm.reshape(-1)

def write_mem(path, data):
    with open(path, "w") as f:
        for v in data:
            if data.dtype == np.int16:
                f.write(f"{int(v) & 0xFFFF:04X}\n")
            elif data.dtype == np.int32:
                f.write(f"{int(v) & 0xFFFFFFFF:08X}\n")
            elif data.dtype == np.float32:
                bits = np.frombuffer(
                    np.float32(v).tobytes(), dtype=np.uint32
                )[0]
                f.write(f"{bits:08X}\n")



In [466]:
containers = {
    part: {
        "weight": {"data": [], "offsets": {}, "cursor": 0},
        "bias":   {"data": [], "offsets": {}, "cursor": 0}
    }
    for part in ["encoder", "bottleneck", "decoder", "out"]
}

bias_layer_counter = {
    "encoder": 0,
    "bottleneck": 0,
    "decoder": 0,
    "out": 0
}

ref_param_sum = 0

# ---- Global statistics (NO bias repetition) ----
w_min = +np.inf
w_max = -np.inf
w_sum = 0
w_cnt = 0

b_min = +np.inf
b_max = -np.inf
b_sum = 0
b_cnt = 0

In [467]:
for name, param in G_fused.state_dict().items():
    arr_f = param.cpu().numpy().astype(np.float32)
    part = classify_part(name) # "encoder", "bottneck", "decoder", or "out"
    wtype = "bias" if is_bias(name) else "weight"

    # Quantize or float
    if MODE == "FLOAT":
        arr = arr_f.astype(np.float32)
        dtype_str = "float32"
        meta = {}
    else:
        arr = float_to_q(
            arr_f,
            frac_bits=FRAC_BITS,
            int_bits=INT_BITS,
            dtype=DTYPE
        )
        dtype_str = TYPE
        meta = {"frac_bits": FRAC_BITS, "int_bits": INT_BITS}

    # ---- Apply ordering rules ----
    if wtype == "weight":
        ref_param_sum += arr.size
        w_min = min(w_min, arr.min())
        w_max = max(w_max, arr.max())
        w_sum += arr.sum()
        w_cnt += arr.size

        if part == "decoder":
            arr_flat = flatten_deconv_weight(arr)
            reorder_note = " (deconv reorder)"
        else:
            arr_flat = flatten_conv_weight(arr)
            reorder_note = ""
    else:
        # bias
        ref_param_sum += arr.size
        b_min = min(b_min, arr.min())
        b_max = max(b_max, arr.max())
        b_sum += arr.sum()
        b_cnt += arr.size

        layer_idx = bias_layer_counter[part]

        if layer_idx >= len(TIME_REPEAT[part]):
            raise RuntimeError(
                f"Too many bias layers in {part}: "
                f"expected {len(TIME_REPEAT[part])}, got more"
            )

        repeat_n = TIME_REPEAT[part][layer_idx]
        bias_layer_counter[part] += 1
        
        arr_flat = expand_bias(arr, repeat_n)
        reorder_note = f" (repeat={repeat_n})"

    # ---- PRINT SIZE CHECK (NEW) ----
    print(
        f"[{part:10}][{wtype:6}] {name:30} -> "
        f"flat_size={arr_flat.size}{reorder_note}"
    )

    c = containers[part][wtype]
    size = arr_flat.size

    c["offsets"][name] = {
        "offset": int(c["cursor"]),
        "shape": list(arr_flat.shape),
        "dtype": dtype_str,
        **meta
    }

    c["data"].append(arr_flat)
    c["cursor"] += size

print("\n===== BIAS LAYER CONSUMPTION CHECK =====")
for part, cnt in bias_layer_counter.items():
    expected = len(TIME_REPEAT[part])
    status = "OK" if cnt == expected else "MISMATCH"
    print(f"{part:10}: used {cnt} / expected {expected}  [{status}]")


[encoder   ][weight] e1.conv.weight                 -> flat_size=512
[encoder   ][bias  ] e1.conv.bias                   -> flat_size=8192 (repeat=256)
[encoder   ][weight] e2.conv.weight                 -> flat_size=32768
[encoder   ][bias  ] e2.conv.bias                   -> flat_size=8192 (repeat=128)
[encoder   ][weight] e3.conv.weight                 -> flat_size=131072
[encoder   ][bias  ] e3.conv.bias                   -> flat_size=8192 (repeat=64)
[encoder   ][weight] e4.conv.weight                 -> flat_size=524288
[encoder   ][bias  ] e4.conv.bias                   -> flat_size=8192 (repeat=32)
[bottleneck][weight] bottleneck.0.0.weight          -> flat_size=458752
[bottleneck][bias  ] bottleneck.0.0.bias            -> flat_size=8192 (repeat=32)
[bottleneck][weight] bottleneck.0.2.weight          -> flat_size=458752
[bottleneck][bias  ] bottleneck.0.2.bias            -> flat_size=8192 (repeat=32)
[bottleneck][weight] bottleneck.1.0.weight          -> flat_size=458752
[bottl

In [468]:
base_dir = os.path.abspath(
    os.path.join(data_path, "..",
    f"Test_G_d{DATA_MODE}_{TYPE}")
)
os.makedirs(base_dir, exist_ok=True)

In [469]:
for part in containers:
    for wtype in ["weight", "bias"]:
        c = containers[part][wtype]
        data = np.concatenate(c["data"])

        json_path = os.path.join(
            base_dir,
            f"G_d{DATA_MODE}_{TYPE}_{part}_{wtype}.json"
        )
        mem_path = os.path.join(
            base_dir,
            f"G_d{DATA_MODE}_{TYPE}_{part}_{wtype}.mem"
        )

        with open(json_path, "w") as f:
            json.dump(c["offsets"], f, indent=2)

        write_mem(mem_path, data)

        print(f"Saved {part}/{wtype}: {data.size}")

Saved encoder/weight: 688640
Saved encoder/bias: 32768
Saved bottleneck/weight: 3670016
Saved bottleneck/bias: 65536
Saved decoder/weight: 217088
Saved decoder/bias: 32768
Saved out/weight: 112
Saved out/bias: 512


# Mem Import

In [470]:
def read_mem_fixed(path, dtype):
    with open(path, "r") as f:
        lines = [l.strip() for l in f if l.strip()]

    hex_arr = np.array(lines)

    if dtype == np.int16:
        # np.uint16 captures the bits, .view(np.int16) handles the negative sign
        val_arr = np.array([int(h, 16) for h in hex_arr], dtype=np.uint16).view(np.int16)
    elif dtype == np.int32:
        val_arr = np.array([int(h, 16) for h in hex_arr], dtype=np.uint32).view(np.int32)
    else:
        raise ValueError("dtype must be np.int16 or np.int32")

    return hex_arr, val_arr

In [471]:
decoder_weight_mem = os.path.join(
    base_dir,
    f"G_d{DATA_MODE}_{TYPE}_decoder_weight.mem"
)

w_hex, w_q = read_mem_fixed(
    decoder_weight_mem,
    dtype=DTYPE   # np.int16 or np.int32
)

print("Decoder weight:")
print("  hex shape :", w_hex.shape)
print("  q   shape :", w_q.shape)

decoder_bias_mem = os.path.join(
    base_dir,
    f"G_d{DATA_MODE}_{TYPE}_decoder_bias.mem"
)

b_hex, b_q = read_mem_fixed(
    decoder_bias_mem,
    dtype=DTYPE
)

print("Decoder bias:")
print("  hex shape :", b_hex.shape)
print("  q   shape :", b_q.shape)


Decoder weight:
  hex shape : (217088,)
  q   shape : (217088,)
Decoder bias:
  hex shape : (32768,)
  q   shape : (32768,)


# Result

## Original

In [472]:
# Decoder last output 
G.eval()
decoder_last_input_batched = decoder_last_input.unsqueeze(0)

with torch.no_grad():
    out_deconv_ref = G.d4.deconv(decoder_last_input_batched)
    out_bn_ref     = G.d4.norm(out_deconv_ref)
    out_d4_ref_original     = G.d4.act(out_bn_ref)

out_2d_original = out_d4_ref_original.squeeze(0)
out_f_original = out_2d_original.detach().cpu().numpy().astype(np.float32)

In [473]:
# float32 input
inp_f = decoder_last_input.detach().cpu().numpy().astype(np.float32)

# quantize to Q9.14
inp_q = float_to_q(
    inp_f,
    frac_bits=14,
    int_bits=10,
    dtype=np.int32
)

out_q_original = float_to_q(
    out_f_original,
    frac_bits=14,
    int_bits=10,
    dtype=np.int32
)


In [474]:
print(inp_q)
print(out_q_original)

[[ 10389 -20070    377 ...   3623   4751  -2806]
 [ -6728 -19803  28842 ...  -7025  -7435  14789]
 [ -6326   5797  13122 ...   9767 -27894  -5219]
 ...
 [  6566  -6092 -20761 ...  20464   8695 -20028]
 [-11184  10882  17755 ...   8395  -5962  12613]
 [-12396   -255 -14755 ... -28268    749   7425]]
[[  542  4028     0 ...     0  3798 14952]
 [ 3118     0    77 ...     0  4133     0]
 [    0     0  3371 ...  7146     0   157]
 ...
 [    0     0 13750 ...  7376  4594     0]
 [ 9272  5095     0 ...     0     0     0]
 [ 1178     0  6759 ...     0  2655 13680]]


## Fused

In [475]:
# Decoder last output 
G.eval()
decoder_last_input_batched = decoder_last_input.unsqueeze(0)

with torch.no_grad():
    out_deconv_ref = G_fused.d4.deconv(decoder_last_input_batched)
    out_d4_ref_fused     = G_fused.d4.act(out_bn_ref)

out_2d_fused = out_d4_ref_fused.squeeze(0)
out_f_fused = out_2d_fused.detach().cpu().numpy().astype(np.float32)

In [476]:
# float32 input
inp_f = decoder_last_input.detach().cpu().numpy().astype(np.float32)

# quantize to Q9.14
inp_q = float_to_q(
    inp_f,
    frac_bits=14,
    int_bits=10,
    dtype=np.int32
)

out_q_fused = float_to_q(
    out_f_fused,
    frac_bits=14,
    int_bits=10,
    dtype=np.int32
)


In [477]:
print(inp_q)
print(out_q_fused)

[[ 10389 -20070    377 ...   3623   4751  -2806]
 [ -6728 -19803  28842 ...  -7025  -7435  14789]
 [ -6326   5797  13122 ...   9767 -27894  -5219]
 ...
 [  6566  -6092 -20761 ...  20464   8695 -20028]
 [-11184  10882  17755 ...   8395  -5962  12613]
 [-12396   -255 -14755 ... -28268    749   7425]]
[[  542  4028     0 ...     0  3798 14952]
 [ 3118     0    77 ...     0  4133     0]
 [    0     0  3371 ...  7146     0   157]
 ...
 [    0     0 13750 ...  7376  4594     0]
 [ 9272  5095     0 ...     0     0     0]
 [ 1178     0  6759 ...     0  2655 13680]]


## Mem derived

In [478]:
# from your JSON / export
D4_W_OFFSET = 212992
D4_W_SIZE   = 4096   # 64 * 1 * 64
D4_BIAS_OFFSET = 24576
TIME_REPEAT_D4 = 512
Cout        = 16
Cin         = 64
K           = 4

# weights (already deconv-flattened in export)
d4_w_flat = w_q[-D4_W_SIZE:]
d4_w_reshaped = d4_w_flat.reshape(Cout, K, Cin)

# bias (expanded during export → just take first Cout)
d4_b_first = b_q[D4_BIAS_OFFSET:D4_BIAS_OFFSET+Cout-1]   # [cout]
d4_b_sliced = np.array([
    b_q[D4_BIAS_OFFSET + oc * TIME_REPEAT_D4]
    for oc in range(Cout)
])

print("d4 weight shape:", d4_w_reshaped.shape)
print("d4 bias shape  :", d4_b_sliced.shape)

d4 weight shape: (16, 4, 64)
d4 bias shape  : (16,)


In [479]:
base_val = b_q[D4_BIAS_OFFSET]
diff = False

for i in range(D4_BIAS_OFFSET,D4_BIAS_OFFSET+TIME_REPEAT_D4):
    if b_q[i] != base_val:
        print(f"Not the same at i = {i}")
        diff = True
        break

if not diff:
    print("All values are the same")

print(F"Last index: {base_val == b_q[D4_BIAS_OFFSET+TIME_REPEAT_D4]}")


All values are the same
Last index: False


In [480]:
input_f = decoder_last_input.detach().cpu().numpy().astype(np.float32)
print(input_f.shape)

arr_q = float_to_q(
        input_f,
        frac_bits=FRAC_BITS,
        int_bits=INT_BITS,
        dtype=DTYPE      # np.int32
    )
print(arr_q.shape)

X, Y = arr_q.shape

hex_arr = []
for y in range(Y): # x then y for decoder
    for x in range(X):
        v = int(arr_q[x, y]) & 0xFFFFFFFF
        hex_arr.append(f"{v:08X}\n")

val_arr = np.array(
            [(int(h, 16) & 0xFFFFFFFF) for h in hex_arr],
            dtype=np.uint32
        ).view(np.int32)

(64, 256)
(64, 256)


In [481]:
Cin, Tin = decoder_last_input.shape
Cout = 16
K = 4
stride = 2
padding = 1

# output length
Tout = (Tin - 1) * stride - 2 * padding + K # (256 - 1) * 2 - 2 * 1 + 4 = 512

# output buffer
out_int = np.zeros((Cout, Tout), dtype=np.int32)

# ---- ConvTranspose1d math ----

for cout in range(Cout):
    for t_out in range(Tout):
        acc = int(d4_b_sliced[cout]) << FRAC_BITS

        for k in range(K):
            t_in_val = t_out + padding - k
            if t_in_val % stride != 0:
                continue
            t_in = t_in_val // stride
            if t_in < 0 or t_in >= Tin:
                continue

            for cin in range(Cin):
                idx = (t_in * Cin) + cin

                weight = d4_w_reshaped[cout, k, cin]
                acc += weight * val_arr[idx]

        rounding_factor = 1 << (FRAC_BITS - 1)
        final_val = (acc + rounding_factor) >> FRAC_BITS
        out_int[cout, t_out] = final_val

In [482]:
print(out_int)

[[   540   4028  -7946 ...  -3355   3800  14946]
 [  3114   -585     79 ...  -2533   4126   -160]
 [ -9749  -1488   3366 ...   7148 -11849    156]
 ...
 [ -1618 -12669  13750 ...   7378   4598  -7433]
 [  9268   5089 -12069 ...  -6294  -8993  -9798]
 [  1183  -9179   6757 ...  -2869   2648  13679]]
